In [66]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time
import hashlib

In [67]:
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")

service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.daraz.com.np/laptops/")

In [68]:
links = []
for i in range(0, 41):
    products = driver.find_elements(By.XPATH, '//*[@id="id-a-link"]')
    urls = [x.get_property("href") for x in products]
    links += urls
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1500);")
    next = driver.find_element(
        By.XPATH,
        '//*[@id="root"]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[9]/a',
    )
    next.click()
    time.sleep(1)

In [71]:
def generate_filename(url):
    hashed_url = hashlib.sha256(url.encode()).hexdigest()
    truncated_hash = hashed_url[:10]
    return truncated_hash 

files = [generate_filename(url) for url in links]
files = [files[i-1] + "_" + str(i) + ".csv" for i in range(1,len(files)+1)]

In [73]:
df = pd.DataFrame(list(zip(links, files)), columns=['Url', 'Filename'])
df.to_csv('../datasets/laptops_url.csv', index=False)